In [ ]:
import os
import sys
import nest_asyncio
nest_asyncio.apply()
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
from section.privacy.privacy_llm import pipeline
base_dir = "./test/privacy/"

os.makedirs(base_dir, exist_ok=True)

pipeline.pipeline(base_dir=base_dir)

In [ ]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
base_dir = "./test/privacy/"

from responses import process_datafolder 
import shutil
source_config = "../section/privacy/privacy_llm/file_config.json"
target_config = os.path.join(base_dir,"final","file_config.json")

if os.path.exists(source_config):
    shutil.copy2(source_config, target_config)
    print(f"Successfully copied file_config to {target_config}")
else:
    print("Warning: Source file_config not found")
    
data_folder  = os.path.join(base_dir,"final")
# model_list = [
#     'gpt-4o', 'gpt-4o-mini', 'gpt-3.5-turbo',
#     'claude-3.5-sonnet', 'claude-3-haiku',
#     'gemini-1.5-pro', 'gemini-1.5-flash', 'gemma-2-27B',
#     'llama-3.1-70B', 'llama-3.1-8B',
#     'glm-4-plus', 'qwen-2.5-72B', 
#     'mistral-8x7B', 'mistral-8x22B',
#     'yi-lightning', 'deepseek-chat']
model_list = [
    'gpt-4o']

async def process_files_original():
    request_types = ['llm',]
    prompt_keys,result_keys = ['prompt'], ['responses']
    for request_type, prompt_key,result_key in zip(request_types, prompt_keys, result_keys):
        await process_datafolder(
            data_folder, request_type=request_type, async_list=model_list, sync_list=[],
            prompt_key=prompt_key, result_key=result_key,file_name_extension='_responses'
        )
    print("All files processed.")
await process_files_original()

In [ ]:
from src.evaluation.judge import JudgeProcessor

config_path = '../src/config/judge_prompt.yaml'
async_judge_model = ['gpt-4o-mini']
response_key = ['responses']
judge_key = 'judge'
judge_type='llm'
model_list = [
    'gpt-4o']
processor = JudgeProcessor(
    folder_path=data_folder,config_path=config_path, 
    async_judge_model=async_judge_model, 
    response_key=response_key, judge_key=judge_key,
    target_async_model=model_list, target_sync_model=[],
    response_extension='_responses',judge_extension='_judge',
    judge_type=judge_type
)

await processor.process_files(max_concurrent_tasks=5)